# Save target-pathway lists as spark dfs and filter out non-gene targets

Input folder: Pathwaganda/data/GSEA_output

In [7]:
from pyspark.sql import SparkSession
import os
import shutil
from pyspark.sql.functions import split, explode, collect_list, col, concat_ws

In [8]:
spark = SparkSession.builder.getOrCreate()

In [4]:
df_test = spark.read.parquet("/Users/polina/Pathwaganda/data/GSEA_output/Reactome_Pathways_2025_diy/diseaseId=EFO_0000094")

In [5]:
df_test.show(5, truncate=False)

+--------------------------------------------------------------------------+-------------+-------------------+------------------+---------------------+-------------------+-------------------+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Target-pathway matrix

Explode targets from propagated_edge and create boolean matrix TxP.

In [3]:
# Base directories - update these paths as needed
t_input_base = "/Users/polina/Pathwaganda/data/GSEA_output/Reactome_Pathways_2025_diy"
t_output_base = "/Users/polina/Pathwaganda/data/target-pathway_matrix_opt/Reactome_Pathways_2025_diy"

In [4]:
def process_parquet_files(input_dir, output_dir):

    # Loop through all disease subdirectories
    for disease_folder in os.listdir(input_dir):
        disease_input_path = os.path.join(input_dir, disease_folder)
        if not os.path.isdir(disease_input_path):
            continue

        # Find the first Parquet file in the disease folder
        parquet_files = [f for f in os.listdir(disease_input_path) if f.endswith(".parquet")]
        if not parquet_files:
            print(f"No Parquet file found in {disease_input_path}")
            continue

        input_parquet_path = os.path.join(disease_input_path, parquet_files[0])

        # Read the Parquet file
        df = spark.read.parquet(input_parquet_path)

        # Select and process relevant columns
        processed_df = (
            df.select("ID", "propagated_edge")
              .withColumn("approvedSymbol", explode(split(col("propagated_edge"), ",")))
              .groupBy("approvedSymbol")
              .agg(concat_ws(",", collect_list("ID")).alias("ID"))
        )

        # Create the corresponding output path
        disease_output_path = os.path.join(output_dir, disease_folder)
        os.makedirs(disease_output_path, exist_ok=True)

        output_parquet_path = os.path.join(disease_output_path)

        # Write the result
        processed_df.write.mode("overwrite").parquet(output_parquet_path)
        print(f"Saved processed file to: {output_parquet_path}")

    spark.stop()


In [5]:
process_parquet_files(t_input_base, t_output_base)

Saved processed file to: /Users/polina/Pathwaganda/data/target-pathway_matrix_opt/Reactome_Pathways_2025_diy/diseaseId=EFO_0000503
Saved processed file to: /Users/polina/Pathwaganda/data/target-pathway_matrix_opt/Reactome_Pathways_2025_diy/diseaseId=EFO_0011015
Saved processed file to: /Users/polina/Pathwaganda/data/target-pathway_matrix_opt/Reactome_Pathways_2025_diy/diseaseId=MONDO_0000569
Saved processed file to: /Users/polina/Pathwaganda/data/target-pathway_matrix_opt/Reactome_Pathways_2025_diy/diseaseId=MONDO_0003916
Saved processed file to: /Users/polina/Pathwaganda/data/target-pathway_matrix_opt/Reactome_Pathways_2025_diy/diseaseId=EFO_0004533
Saved processed file to: /Users/polina/Pathwaganda/data/target-pathway_matrix_opt/Reactome_Pathways_2025_diy/diseaseId=MONDO_0002033
Saved processed file to: /Users/polina/Pathwaganda/data/target-pathway_matrix_opt/Reactome_Pathways_2025_diy/diseaseId=MONDO_0017343
Saved processed file to: /Users/polina/Pathwaganda/data/target-pathway_matr

In [9]:
spark.read.parquet("/Users/polina/Pathwaganda/data/target-pathway_matrix_opt/Reactome_Pathways_2025_diy/diseaseId=EFO_0000094").show(5)

+----------------+--------------------+
|  approvedSymbol|                  ID|
+----------------+--------------------+
| complete genome|R-HSA-1643685,R-H...|
|        18S rRNA|       R-HSA-1643685|
|              1B|R-HSA-1643685,R-H...|
|              1C|R-HSA-1643685,R-H...|
|              1a|R-HSA-1643685,R-H...|
+----------------+--------------------+
only showing top 5 rows


# Pathway embeddings

## Hierarchical (Poincare ball model)

## Weighted (Jaccard similarity index)

# Target embeddings

## Hierarchical (Poincare coordinates)

Based on each file create pathway coordinates in hyperbolic space.

In [3]:
# Target info from OT platform to parse gene names
target_path = "gs://open-targets-data-releases/25.03/output/target/"
target = spark.read.parquet(target_path)
target_names = target.select("approvedSymbol", "id")

In [4]:
# Rename columns and aggregate by approvedSymbol
target_names = target.select("approvedSymbol", "id").groupBy('approvedSymbol').agg(
    f.concat_ws(',', f.collect_list('id')).alias('targetId')
)

In [5]:
def calculate_propagated_targets_with_terms(
    input_gcs_dir, 
    output_gcs_dir, 
    folders_to_process, 
    target_table
):
    """
    Process CSV files in specified GCS folders to generate Spark DataFrames
    with propagated targets and their corresponding terms. Merges with target_table
    on approvedSymbol and filters out unmatched rows. Saves results in Parquet format.

    Args:
        input_gcs_dir (str): Input GCS directory path (gs://bucket/path/)
        output_gcs_dir (str): Output GCS directory path (gs://bucket/path/)
        folders_to_process (list): List of folder names to process
        target_table (DataFrame): Spark DataFrame with 'approvedSymbol' and 'targetId'
    """
    input_gcs_dir = input_gcs_dir.rstrip("/")
    output_gcs_dir = output_gcs_dir.rstrip("/")

    for folder_name in folders_to_process:
        csv_files = spark.sparkContext.wholeTextFiles(f"{input_gcs_dir}/{folder_name}/*.csv").keys().collect()
        
        for csv_file in csv_files:
            csv_name = csv_file.split("/")[-1].replace(".csv", "")
            output_folder_path = f"{output_gcs_dir}/{folder_name}/{csv_name}"

            df = spark.read.option("header", True).csv(csv_file)

            if not {'propagated_edge', 'Term'}.issubset(df.columns):
                print(f"Skipping {csv_file}: missing required columns.")
                continue

            df_exploded = df.withColumn(
                "approvedSymbol", explode(split(col("propagated_edge"), ","))
            ).dropna(subset=["approvedSymbol"])

            target_terms_df = df_exploded.groupBy("approvedSymbol") \
                .agg(collect_set("Term").alias("terms"))

            merged_df = target_terms_df.join(
                target_table.select("approvedSymbol", "targetId"),
                on="approvedSymbol",
                how="inner"
            )

            # Write as Parquet (supports array columns)
            merged_df.write.mode("overwrite").parquet(output_folder_path)
            print(f"Saved: {output_folder_path}")

In [15]:
calculate_propagated_targets_with_terms(
    input_gcs_dir = "gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output", 
    output_gcs_dir = "gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark", 
    folders_to_process = ["KEGG_2021_Human"], 
    target_table = target_names
)

Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/KEGG_2021_Human/EFO_0000095_ge_mm_som_gsea_KEGG_2021_Human_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/KEGG_2021_Human/EFO_0000222_ge_mm_som_gsea_KEGG_2021_Human_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/KEGG_2021_Human/EFO_0000274_ge_mm_gsea_KEGG_2021_Human_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/KEGG_2021_Human/EFO_0000183_ge_mm_som_gsea_KEGG_2021_Human_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/KEGG_2021_Human/EFO_0000341_ge_mm_gsea_KEGG_2021_Human_pval0.05


...

Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/KEGG_2021_Human/MONDO_0008170_ge_mm_som_gsea_KEGG_2021_Human_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/KEGG_2021_Human/MONDO_0008315_ge_mm_som_gsea_KEGG_2021_Human_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/KEGG_2021_Human/MONDO_0011719_ge_mm_som_gsea_KEGG_2021_Human_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/KEGG_2021_Human/EFO_0000729_ge_mm_gsea_KEGG_2021_Human_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/KEGG_2021_Human/EFO_0000756_ge_mm_som_gsea_KEGG_2021_Human_pval0.05


In [6]:
calculate_propagated_targets_with_terms(
    input_gcs_dir = "gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output", 
    output_gcs_dir = "gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark", 
    folders_to_process = ["Reactome_Pathways_2024"], 
    target_table = target_names
)

Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/Reactome_Pathways_2024/EFO_0000095_ge_mm_som_gsea_Reactome_Pathways_2024_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/Reactome_Pathways_2024/EFO_0000222_ge_mm_som_gsea_Reactome_Pathways_2024_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/Reactome_Pathways_2024/EFO_0000274_ge_mm_gsea_Reactome_Pathways_2024_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/Reactome_Pathways_2024/EFO_0000275_ge_mm_gsea_Reactome_Pathways_2024_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/Reactome_Pathways_2024/EFO_0000341_ge_mm_gsea_Reactome_Pathways_2024_pval0.05


...

Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/Reactome_Pathways_2024/MONDO_0005301_ge_mm_gsea_Reactome_Pathways_2024_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/Reactome_Pathways_2024/MONDO_0007915_ge_mm_gsea_Reactome_Pathways_2024_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/Reactome_Pathways_2024/MONDO_0008170_ge_mm_som_gsea_Reactome_Pathways_2024_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/Reactome_Pathways_2024/MONDO_0008315_ge_mm_som_gsea_Reactome_Pathways_2024_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/Reactome_Pathways_2024/MONDO_0011719_ge_mm_som_gsea_Reactome_Pathways_2024_pval0.05


In [8]:
calculate_propagated_targets_with_terms(
    input_gcs_dir = "gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output", 
    output_gcs_dir = "gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark", 
    folders_to_process = ["WikiPathways_2024_Human"], 
    target_table = target_names
)

Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/WikiPathways_2024_Human/EFO_0000095_ge_mm_som_gsea_WikiPathways_2024_Human_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/WikiPathways_2024_Human/EFO_0000222_ge_mm_som_gsea_WikiPathways_2024_Human_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/WikiPathways_2024_Human/EFO_0000183_ge_mm_som_gsea_WikiPathways_2024_Human_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/WikiPathways_2024_Human/EFO_0000274_ge_mm_gsea_WikiPathways_2024_Human_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/WikiPathways_2024_Human/EFO_0000341_ge_mm_gsea_WikiPathways_2024_Human_pval0.05


...

Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/WikiPathways_2024_Human/MONDO_0007915_ge_mm_gsea_WikiPathways_2024_Human_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/WikiPathways_2024_Human/MONDO_0008170_ge_mm_som_gsea_WikiPathways_2024_Human_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/WikiPathways_2024_Human/MONDO_0008315_ge_mm_som_gsea_WikiPathways_2024_Human_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/WikiPathways_2024_Human/MONDO_0011719_ge_mm_som_gsea_WikiPathways_2024_Human_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/WikiPathways_2024_Human/EFO_0000756_ge_mm_som_gsea_WikiPathways_2024_Human_pval0.05


In [9]:
calculate_propagated_targets_with_terms(
    input_gcs_dir = "gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output", 
    output_gcs_dir = "gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark", 
    folders_to_process = ["GO_Biological_Process_2023"], 
    target_table = target_names
)

Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/GO_Biological_Process_2023/EFO_0000095_ge_mm_som_gsea_GO_Biological_Process_2023_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/GO_Biological_Process_2023/EFO_0000183_ge_mm_som_gsea_GO_Biological_Process_2023_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/GO_Biological_Process_2023/EFO_0000222_ge_mm_som_gsea_GO_Biological_Process_2023_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/GO_Biological_Process_2023/EFO_0000274_ge_mm_gsea_GO_Biological_Process_2023_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/GO_Biological_Process_2023/EFO_0000275_ge_mm_gsea_GO_Biological_Process_2023_pval0.05


...

Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/GO_Biological_Process_2023/MONDO_0007915_ge_mm_gsea_GO_Biological_Process_2023_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/GO_Biological_Process_2023/MONDO_0008170_ge_mm_som_gsea_GO_Biological_Process_2023_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/GO_Biological_Process_2023/MONDO_0008315_ge_mm_som_gsea_GO_Biological_Process_2023_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/GO_Biological_Process_2023/MONDO_0011719_ge_mm_som_gsea_GO_Biological_Process_2023_pval0.05


Saved: gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/GO_Biological_Process_2023/EFO_0000756_ge_mm_som_gsea_GO_Biological_Process_2023_pval0.05


In [10]:
spark_df = spark.read.parquet("gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output_spark/GO_Biological_Process_2023/EFO_0000756_ge_mm_som_gsea_GO_Biological_Process_2023_pval0.05")

In [11]:
spark_df.show(10, truncate=False)

+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|approvedSymbol|terms                                                                                                                                                                                                                                                              |targetId       |
+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|AATK          |[Protein Autophosphorylation (GO:0046777)]                                                               

In [12]:
spark_df.count()

8959

# Save target-pathways lists for tensor board visualisation

! This code is pasted from IBD_pathway_to_cell from similarity_mvp and haven't been run !

Process CSV files in specified folders within a GCS directory and save target-pathway relationships in a format suitable for TensorBoard Embedding Projector. 

In [ ]:
def save_target_pathway_tensorboard(input_gcs_dir, output_gcs_dir, folders_to_process):
    """
    Args:
        input_gcs_dir (str): Input GCS directory path.
        output_gcs_dir (str): Output GCS directory path.
        folders_to_process (list): List of folder names within the input directory to process.

    Output:
        Saves target-pathway relationships as TSV files in the output GCS directory.
    """
    # Initialize GCS filesystem
    fs = gcsfs.GCSFileSystem()

    # Ensure no trailing slashes in input and output directories
    input_gcs_dir = input_gcs_dir.rstrip("/")
    output_gcs_dir = output_gcs_dir.rstrip("/")

    for folder_name in folders_to_process:
        folder_path = f"{input_gcs_dir}/{folder_name}"
        output_folder_path = f"{output_gcs_dir}/{folder_name}"

        # Ensure output folder exists
        if not fs.exists(output_folder_path):
            fs.mkdirs(output_folder_path)

        # List files in the input folder
        files_in_folder = fs.ls(folder_path)
        csv_files = [file for file in files_in_folder if file.endswith('.csv')]

        for file_path in csv_files:
            # Read the CSV file directly from GCS
            with fs.open(file_path, 'r') as f:
                df = pd.read_csv(f)

            # Check if 'propagated_edge' and 'Term' exist in the file
            if 'propagated_edge' not in df.columns or 'Term' not in df.columns:
                print(f"Skipping {file_path}: missing required columns.")
                continue

            # Explode the `propagated_edge` column
            df['propagated_edge_exploded'] = df['propagated_edge'].str.split(',')
            df = df.explode('propagated_edge_exploded').dropna(subset=['propagated_edge_exploded'])

            # Prepare data for TensorBoard format
            embedding_metadata = df[['propagated_edge_exploded', 'Term']]
            embedding_metadata = embedding_metadata.rename(
                columns={"propagated_edge_exploded": "Target", "Term": "Pathway"}
            )

            # Save target-pathway relationships as a TSV
            output_file_name = f"{Path(file_path).stem}_target_pathway.tsv"
            output_file_path = f"{output_folder_path}/{output_file_name}"

            # Save directly to GCS
            with fs.open(output_file_path, 'w') as f:
                embedding_metadata.to_csv(f, sep='\t', index=False)

            print(f"Processed and uploaded: {output_file_path}")


In [ ]:
input_gcs_dir = "gs://ot-team/polina/pathway_propagation_validation_v2/gsea_output"
output_gcs_dir = "gs://ot-team/polina/pathway_propagation_validation_v2/4tensorboard/jaccard"

library = ["Reactome_Pathways_2024"]

save_target_pathway_tensorboard(input_gcs_dir, output_gcs_dir, library)